<a href="https://colab.research.google.com/github/kgautam2103/Meetup_automation/blob/main/delete_events_meetup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# when you run this code, you will get a link below
#Follow the link, copy the code, paste it in the box and then press Enter on the keyboard.
#This will authenticate you and will allow you to interact with Google Sheets and other Google apps in your notebook.
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
client_key=""

client_redirect_url="https://us.sahajayoga.org/"

authorize_url = "https://secure.meetup.com/oauth2/authorize?client_id="+client_key+"&response_type=code&redirect_uri="+client_redirect_url

print(authorize_url)

https://secure.meetup.com/oauth2/authorize?client_id=emgo5alo5672bdgr1k4bp6sn57&response_type=code&redirect_uri=https://us.sahajayoga.org/


In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from dateutil import tz

def get_access_refresh_token_from_code(access_code,client_secret, grant_type_code):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_code+"&redirect_uri="+client_redirect_url+"&code="+access_code


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


def get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh):
  access_url = "https://secure.meetup.com/oauth2/access?client_id="+client_key+"&client_secret="+client_secret+"&grant_type="+grant_type_refresh+"&refresh_token="+refresh_token


  response = requests.post(access_url)

  print(response.json())

  access_token = response.json()["access_token"]
  refresh_token = response.json()["refresh_token"]

  return access_token,refresh_token


access_code = input("please enter the access code recieved ")

client_secret=""

grant_type_code = "authorization_code"

grant_type_refresh = "refresh_token"

meetup_api_url = "https://api.meetup.com/gql"

#sheet_name = input('enter the start time in UTC in format 2022-12-31T13:45 : ')

access_token,refresh_token = get_access_refresh_token_from_code(access_code,client_secret, grant_type_code)

please enter the access code recieved 586470b72f8be5f4c904054d63ce1889
{'access_token': '385991dbb0e2b854598eb33eab1adc48', 'refresh_token': 'f0fbf5955c7971693da7eae7d69bfda6', 'token_type': 'bearer', 'expires_in': 3600}


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1uUDCnrPDJJUbZkUQu8YT8XDiv_ObSq5DXaTFKOQQEh0/edit#gid=0')

current_ts = datetime.now().strftime('%Y%m%d%H%M%S')
output_sheet = current_ts+'_event_delete_output_meetup'

counter=1

# create and Open our new sheet and add some data.
sh = gc.create(output_sheet)
output_sheet = gc.open(output_sheet).sheet1

output_sheet.insert_row(['event_id','event_url','is_deleted','errors'],1)

sheet_name = input('enter the name of input sheet : ')
input_sheet = wb.worksheet(sheet_name)
inputdata = input_sheet.get_all_values()
inputdata_df = pd.DataFrame(inputdata, columns=['event_id','event_url'])
inputdata_df = inputdata_df.drop(inputdata_df.index[0])

for index,row in inputdata_df.iterrows():
  counter = counter+1
  is_deleted=""
  delete_event_error=""
  access_token,refresh_token = get_access_refresh_token_from_refresh(refresh_token,client_secret, grant_type_refresh)
  authorization = 'Bearer ' + access_token
  auth_headers = {
    'Authorization': authorization,
    'Content-Type': 'application/json'
    }

  print("access_token "+access_token)
  print("refresh_token "+refresh_token)
  cell_value_list = []
  event_id = row['event_id']
  event_url = row['event_url']

  print(event_id)

  delete_event_query = """mutation($delete_event_input: DeleteEventInput!) {
  deleteEvent(input: $delete_event_input) {
      success
      errors {
        message
        code
        field
      }
    }
  }"""

  delete_event_variables = {
    "delete_event_input": {
      "eventId": event_id
    }
  }

  delete_event_mutation_response = requests.post(meetup_api_url, json={'query': delete_event_query, 'variables': delete_event_variables}, headers=auth_headers)

  print(delete_event_mutation_response.text)

  is_deleted = delete_event_mutation_response.json()['data']['deleteEvent']['success']
  try:
    delete_event_error = delete_event_mutation_response.json()['data']['deleteEvent']['errors'][0]['message']
  except:
    delete_event_error=""

  cell_value_list = [event_id,event_url,is_deleted,delete_event_error]
  output_sheet.insert_row(cell_value_list,counter)

enter the name of input sheet : inputdata
{'access_token': '8351241da8062790b5c934c334ad7956', 'refresh_token': 'f0fbf5955c7971693da7eae7d69bfda6', 'token_type': 'bearer', 'expires_in': 3600}
access_token 8351241da8062790b5c934c334ad7956
refresh_token f0fbf5955c7971693da7eae7d69bfda6
290187241
{"data":{"deleteEvent":{"success":true,"errors":[]}}}

